# Imports

In [1]:
import pandas as pd
from sklearn.feature_extraction import DictVectorizer
from sklearn.linear_model import LinearRegression
import seaborn as sns
import matplotlib as plt
from sklearn.metrics import mean_squared_error

# Question 1

In [2]:
df = pd.read_parquet("yellow_tripdata_2023-01.parquet")

In [3]:
df.shape

(3066766, 19)

# Question 2

In [4]:
df['duration'] = (df['tpep_dropoff_datetime'] - df['tpep_pickup_datetime']).dt.total_seconds() / 60


In [5]:
df['duration'].std()

42.594351241920904

# Question 3

In [6]:
no_outliers_df = df.query('duration >= 1 and duration <= 60')

In [7]:
no_outliers_df.shape[0]/df.shape[0]

0.9812202822125979

# Question 4

In [8]:
train_df = no_outliers_df[['PULocationID', 'DOLocationID']]

In [9]:
categorical = ['PULocationID', 'DOLocationID']

In [10]:
train_df[categorical] = train_df[categorical].astype(str)

/tmp/ipykernel_8551/2611350797.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  train_df[categorical] = train_df[categorical].astype(str)


In [11]:
train_df[categorical].dtypes

PULocationID    object
DOLocationID    object
dtype: object

In [12]:
train_dict = train_df.to_dict(orient='records')

In [13]:
dv = DictVectorizer()
X_train = dv.fit_transform(train_dict)

In [14]:
X_train

<3009173x515 sparse matrix of type '<class 'numpy.float64'>'
	with 6018346 stored elements in Compressed Sparse Row format>

# Question 5

In [15]:
target = 'duration'
y_train = no_outliers_df[target].values

In [16]:
lr = LinearRegression()
lr.fit(X_train, y_train)

LinearRegression()

In [17]:
y_pred = lr.predict(X_train)

In [18]:
mean_squared_error(y_train, y_pred, squared=False)

7.649261027828148

# Question 6

In [27]:
val_df = pd.read_parquet('yellow_tripdata_2023-02.parquet')

In [28]:
val_df['duration'] = (val_df['tpep_dropoff_datetime'] - val_df['tpep_pickup_datetime']).dt.total_seconds() / 60

In [29]:
val_df = val_df.query('duration >= 1 and duration <= 60')

In [30]:
target = 'duration'
y_val = val_df[target].values

In [31]:
val_df[categorical] = val_df[categorical].astype(str)

In [32]:
val_dict = val_df[categorical].to_dict(orient='records')

In [33]:
X_val = dv.transform(val_dict)

In [34]:
y_pred = lr.predict(X_val)

In [35]:
mean_squared_error(y_val, y_pred, squared=False)

7.811832556028926